In [1]:
import nltk
import spacy

nltk.download('punkt')
nltk.download('rslp')

[nltk_data] Downloading package punkt to /home/victorhugo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /home/victorhugo/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [2]:
#importação das bibliotecas criadas para normalização dos dados
from nlputils import lexical
from nlputils import morphosyntax
from nlputils import syntax

In [3]:
#chamada da bibioteca de preprocessamento
lexical_normalizer = lexical.Preprocessing()
morphosyntax_normalizer = morphosyntax.Preprocessing('../models/pt_core_news_sm-2.1.0')
syntax_normalizer = syntax.Preprocessing('../models/pt_core_news_sm-2.1.0')

In [4]:
import os

#definição do diretorio dos corpus e criacao de uma lista com os nomes de cada arquivo dentro do diretorio
corpora_path = '../data/corpora/'
files_corpora = os.listdir(corpora_path)
files_corpora = [d for d in files_corpora if d not in '.DS_Store']

In [16]:
#criacao de um dicionario que ira armazenar cada corpus em uma chave
sentences_dic = {}
all_files = []
for corpus in files_corpora:
    files = [os.path.join(corpora_path + corpus, f) \
             for f in os.listdir(corpora_path + corpus) \
             if os.path.isfile(os.path.join(corpora_path + corpus, f))]
    #cada corpus tera mais 4 chaves para armazenar informacoes de trabalho. Obs.: svo = sujeito, verbo, objeto
    sentences_dic[corpus] = {'sentencas': [], 'tag': [], 'parse': [], 'svo': []}
    
    #adiciona todos os arquivos em uma unica lista independentemente do corpus
    print(len(files))
    all_files.extend(files)
    
    #para cada arquivo em um corpus sera extraido suas frases e armazenadas em cada linha de uma lista
    # limitamos a 30 arquivos pois demora muito para compilar, para fazer o teste com todos é só remover a parte
    # entre colchetes.
    for file in files[:30]:
        with open(file, 'r', encoding='utf-8') as text_file:
            lines = text_file.readlines()
            for line in lines:
                if line != '\n':
                    #toda a sentenca sera escrita em letras minusculas
                    #line = lexical_normalizer.lowercase(line) 
                    #tokeniza as sentencas
                    sentences_line = lexical_normalizer.tokenize_sentences(line)
                    for sentence in sentences_line:
                        #print(sentence)
                        #adiciona cada sentenca de uma linha no dicionario
                        sentences_dic[corpus]['sentencas'].append(sentence)
                        sentence = lexical_normalizer.remove_punctuation(sentence)
                        #adiciona uma lista de cada palavra da sentenca taggeada composta de 
                        #(token, etiqueta_morfossintática)
                        sentences_dic[corpus]['tag'].append(morphosyntax_normalizer.tag(sentence))
                        #adiciona uma lista de cada palavra da sentenca composta de 
                        #(token, papel_sintático, head),
                        sentences_dic[corpus]['parse'].append(syntax_normalizer.parse(sentence))
                        #adiciona uma lista de cada sentenca composta por tuplas de (sujeito, verbo, objeto)
                        sentences_dic[corpus]['svo'].append(syntax_normalizer.get_SVO(sentence))
                    

601
533


In [17]:
sentences_dic.keys()

dict_keys(['educacao', 'tecnologia'])

In [18]:
#utilizacao do Pandas para visualizao dos dados em forma de tabelas
import pandas as pd

#cracao de um dicionario que ira armazenar cada corpus em suas respectivas keys.
dataframes_sentences = {}
for key in sentences_dic.keys():
    #os corpus armazenados aqui estara em formato de DataFrame onde cada key sera uma coluna da tabela
    dataframes_sentences[key] = pd.DataFrame(sentences_dic[key], columns=['sentencas','tag','parse', 'svo'])

## Utilizamos dataframes para mostrar os dados dos córpus
- como em nosso corpora temos os conteudos 'tecnologia' e 'educacao', separamos os dois para serem mostrados abaixo.

In [19]:
dataframes_sentences['tecnologia'].head(n=5)

,sentencas,tag,parse,svo
0,Aparelhos têm bateria de mais de 10 horas e po...,"[(Aparelhos, PROPN), (têm, VERB), (bateria, NO...","[(Aparelhos, nsubj, têm), (têm, ROOT, têm), (b...","[(Aparelhos, têm, bateria), (Charge, são, caix..."
1,"Com o popular design da fabricante, os aparelh...","[(Com, ADP), (o, DET), (popular, ADJ), (design...","[(Com, case, design), (o, det, design), (popul...","[(aparelhos, são, semelhantes), (no, são, seme..."
2,Tanto a Charge 3 quanto a Xtreme estão à venda...,"[(Tanto, ADV), (a, ADP), (Charge, PROPN), (3, ...","[(Tanto, cc, Charge), (a, det, Charge), (Charg...","[(Charge, estão, loja)]"
3,Vale lembrar que já é possível encontrá-las ma...,"[(Vale, VERB), (lembrar, VERB), (que, SCONJ), ...","[(Vale, ROOT, Vale), (lembrar, xcomp, Vale), (...","[(–, acompanham, da)]"
4,"No entanto, as caixas de som têm tamanhos bast...","[(No, ADP), (entanto, NOUN), (as, DET), (caixa...","[(No, case, entanto), (entanto, obl, têm), (as...","[(caixas, têm, entanto), (Xtreme, precisa, 89)]"


In [20]:
dataframes_sentences['educacao'].head(n=5)

,sentencas,tag,parse,svo
0,"A esquerda: o louco, o criminoso e o drogado25...","[(A, DET), (esquerda, NOUN), (o, DET), (louco,...","[(A, det, esquerda), (esquerda, nsubj, crimino...","[(esquerda, é, criminoso), (tudo, é, subprodut..."
1,"), ou voltadas para a educação (orfanatórios, ...","[( , SPACE), (ou, PUNCT), (voltadas, NOUN), (p...","[( , , ou), (ou, cc, voltadas), (voltadas, ROO...",[]
2,nada mais são senão que ‘espaços de opressão’ ...,"[(nada, PRON), (mais, ADV), (são, VERB), (senã...","[(nada, fixed, que), (mais, advmod, nada), (sã...",[]
3,Seu objetivo maior não é a defesa da sociedade...,"[(Seu, DET), (objetivo, NOUN), (maior, ADJ), (...","[(Seu, det, objetivo), (objetivo, nsubj, defes...","[(objetivo, é, defesa), (que, criar, mecanismo..."
4,"Da mesma maneira, o existir do hospício serve ...","[(Da, NOUN), (mesma, DET), (maneira, NOUN), (o...","[(Da, ROOT, Da), (mesma, amod, maneira), (mane...","[(clínicos, excedam, psiquiátricos), (nos, Non..."


## Questão 2:
## Utilizando	 o	 corpora	 compilado	 para	 a	 Prova	 1 e	 as	 rotinas definidas na questão	 anterior, realizar	 a	 extração	 de	 informações	 no	 formado	 de	 triplas:(Sujeito,	Verbo,	Objeto).

### a: Um	dicionário	Python	deve	ser	criado	da	seguinte	forma: a. {“verbo lematizado1”: [(Sujeito1, Objeto1), (Sujeito2,None), ..., (Sujeiton, Objeton)], ..., “verbo lematizadok”: [(Sujeito1, Objeto1), (Sujeito2, Objeto2), ..., (Sujeitom, Objetom)]}

In [21]:
#dicionario contendo lista de verbos e para cada verbo uma lista de tuplas contendo (Sujeito, Objeto)
lemma_verb = {}
#lista de tuplas dos verbos que nao possuem sujeito no formato (verbo lematizado, verbo encontrado)
no_obj_verb = []
#para cada corpus
for corpus in sentences_dic:
    #pegaremos as sentencas em sentences_dic[corpus]['svo']
    for sentence in sentences_dic[corpus]['svo']:
        #pegaremos cada tripla na lista de triplas(sujeito, verbo, objeto) da sentenca
        for svo in sentence:
            #evitar pegar casos vazios
            if svo[1] != None:
                #recebe o verbo lematizado
                verb = svo[1].lemma_
                #verifica se o verbo ainda não foi adicionado no dicionario
                if verb in lemma_verb.keys():
                    #tupla com sujeito e objeto
                    lemma_verb[verb].append((svo[0], svo[2]))
                else:
                    #cria a chave do verbo e adiciona a tupla com sujeito e objeto
                    lemma_verb[verb] = []
                    lemma_verb[verb].append((svo[0], svo[2]))
                #para as sentencas sem objetos, adicionamos o verbo lematizado e o verbo como foi encontrado no texto
                ## na lista de tuplas no_obj_verb, onde guardamos todos os verbos que em algum momento não tiveram objetos.
                if svo[2] == None and svo[2] not in no_obj_verb:
                    no_obj_verb.append((verb, svo[1]))

### b. Exibir as	seguintes	estatísticas:

In [22]:
#salvamos para cada verbo o numero de tuplas (sujeitos, objetos) que ele contem.
stat_verbs = {'verbo': [], 'num_svo': []}
for verb in lemma_verb:
    stat_verbs['verbo'].append(verb)
    stat_verbs['num_svo'].append(len(lemma_verb[verb]))

### i. Qual	verbo	tem	a	maior	lista	de	sujeitos	e	objetos ?

In [23]:
# verbo com o maior numero de lista de sujeitos e objetos.
df = pd.DataFrame(stat_verbs, columns=['verbo','num_svo'])
df.sort_values(by="num_svo").tail(n=1)

,verbo,num_svo
58,ser,251


### ii. Há	algum	verbo	sem	objetos?	Mostre	alguns.

In [24]:
print(len(no_obj_verb))
print(no_obj_verb[:10])

108
[('existir', existe), ('desencadear', desencadeada), ('costumar', costumam), ('monitorados', monitorados), ('encarnar', encarnara), ('apontar', apontou), ('saber', soubessem), ('saber', sabe), ('espalhandose', espalhandose), ('Apolo', Apolo)]


### 3. Dada	uma	pergunta, no formato “O	que/quem verbo?" responde “O que/quem verbo	objeto”.
- Exemplos	de	perguntas:
Quem	pintou	um	quadro?
O	que	derrubou	os	preços?
- Exemplos	de	respostas:
Michelangelo	pintar	quadros.
Notícias	ruins	derrubar	os	preços.

In [25]:
question = input('Faça uma pergunta no formato:  “O que/quem verbo? ”\n ')
#recebe a pergunta normalizada na forma [sujeito:que/quem, verbo, objeto]
ext_data = syntax_normalizer.get_SVO(question)


Faça uma pergunta no formato:  “O que/quem verbo? ”
 quem tem bateria ?


In [26]:

verbo = ext_data[0][1].lemma_
obj = ext_data[0][2]
#fazemos um matching entre o objeto e o verbo e assim encontramos todas as combinações que ambos aparecem
# e adicionamos o sujeito para cada caso.
for tupla in lemma_verb[verbo]:
    if tupla[1] != None and obj.text == tupla[1].text:
        suj = tupla[0]
        print(str(suj) + ' ' + str(verbo) + ' ' + str(obj))

Aparelhos ter bateria
